# Azure OpenAI Python SDK Sample


In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

ModuleNotFoundError: No module named 'openai'

In [52]:
def get_completion(_prompt, model="gpt-35-turbo-deployment"):
    messages = [{"role": "user", "content": _prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content

In [53]:
get_completion("What is the capital of Suriname?")

'Paramaribo.'

In [54]:
customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! \
And to make matter worse, the warranty don't cover the cost of cleaning up me kitchen. \
I need ye help right now, matey!
"""

In [55]:
style = """British English in a calm and respectful tone
"""

In [56]:
prompt = f"""
Translate the text that is delimited by triple backticks into a style that is {style}.\
text: ```{customer_email}```
"""
print(prompt)


Translate the text that is delimited by triple backticks into a style that is British English in a calm and respectful tone
.text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matter worse, the warranty don't cover the cost of cleaning up me kitchen. I need ye help right now, matey!
```



In [57]:
get_completion(prompt)

"I must say, I am quite upset that the lid of my blender detached and consequently caused my kitchen walls to become splattered with smoothie. What's more, the warranty does not cover the expense of me cleaning up the mess in my kitchen. I would greatly appreciate your assistance as soon as possible, my dear friend."

In [82]:
from langchain_openai.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(deployment_name="gpt-35-turbo-deployment")
chat

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10e19f4a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10e1f13d0>, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://alex-openai-trial.openai.azure.com/', deployment_name='gpt-35-turbo-deployment', openai_api_version='2024-02-01', openai_api_type='azure')

In [83]:
template_string = """
Translate the text that is delimited by triple backticks into a style that is {style}.\
text: ```{text}```
"""

## Add a new prompt template

In [84]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='\nTranslate the text that is delimited by triple backticks into a style that is {style}.text: ```{text}```\n')

In [85]:
prompt_template.messages[0].input_variables

['style', 'text']

In [86]:
customer_style = """American English in a calm and respectful tone
"""

In [87]:
customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! \
And to make matter worse, the warranty don't cover the cost of cleaning up me kitchen. \
I need ye help right now, matey!
"""

In [88]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

In [90]:
print(customer_messages[0])

content="\nTranslate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n.text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matter worse, the warranty don't cover the cost of cleaning up me kitchen. I need ye help right now, matey!\n```\n"


In [94]:
customer_response = chat.invoke(customer_messages)
print(customer_response.content)

I am quite upset that my blender lid came off and covered my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance, my friend. Thank you kindly.


## Parsing the output

In [95]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings: \
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It is slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information: 

gift: Was the item purchased as a gift for someone else?
delivery_days: How many days did it take for the product to arrive?
price_value: Extract any sentences about the value or price.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text} 
"""

In [96]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information: \n\ngift: Was the item purchased as a gift for someone else?\ndelivery_days: How many days did it take for the product to arrive?\nprice_value: Extract any sentences about the value or price.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text} \n'))]


In [97]:
messages = prompt_template.format_messages(text=customer_review)
chat = AzureChatOpenAI(deployment_name="gpt-35-turbo-deployment")
response = chat.invoke(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
